In [1]:
import os
import pandas as pd


In [2]:
file1 = "./MDC_Housing_Data.csv"
housing_data = pd.read_csv(file1)

In [3]:
file2 = "./ZIP_TRACT_2019.csv"
zip_vs_tract = pd.read_csv(file2)

In [4]:
file3 = './ACSST5Y2018.S1902_data_with_overlays_2020-01-11T105425.csv'
income_data = pd.read_csv(file3)

In [5]:
housing_data
cols = [0,3,49]
housing_data = housing_data
housing_data = pd.DataFrame(housing_data.iloc[:, cols])


In [6]:
new = housing_data.drop(housing_data.index[[0,2]])
new

,"Miami-Dade County, FL - All Homes",Unnamed: 3,Unnamed: 49
1,Region Name,Current,Current
3,33010,"$321,100",---
4,33012,"$251,200","$1,807"
5,33013,"$341,000",---
6,33014,"$268,800","$1,892"
7,33015,"$271,400","$1,820"
8,33016,"$204,900","$1,673"
9,33018,"$327,400","$2,172"
10,33027,"$317,800","$2,186"
11,33030,"$258,600","$1,816"


In [7]:
new_header = new.iloc[0] 
new = new[1:]
new.columns = new_header 


In [8]:
new.head(25)
col_names = ['zip_code','home_value','rent_value']

In [9]:
new.columns = col_names

In [10]:
home_value_byzip = new
home_value_byzip

,zip_code,home_value,rent_value
3,33010,"$321,100",---
4,33012,"$251,200","$1,807"
5,33013,"$341,000",---
6,33014,"$268,800","$1,892"
7,33015,"$271,400","$1,820"
8,33016,"$204,900","$1,673"
9,33018,"$327,400","$2,172"
10,33027,"$317,800","$2,186"
11,33030,"$258,600","$1,816"
12,33031,"$466,700",---


In [11]:
zip_vs_tract = zip_vs_tract[['zip','tract']]

zip_vs_tract

,zip,tract
0,501,36103158607
1,601,72113071700
2,601,72001956800
3,601,72001956300
4,601,72001956600
5,601,72001956700
6,602,72005400300
7,602,72003430602
8,602,72003430300
9,602,72003430200


In [12]:
zip_data = new.join(zip_vs_tract)
zip_data = zip_data[['zip_code','home_value','rent_value']]
zip_data['zip_code'] = zip_data.zip_code.astype(float)


In [13]:
zip_data

,zip_code,home_value,rent_value
3,33010.0,"$321,100",---
4,33012.0,"$251,200","$1,807"
5,33013.0,"$341,000",---
6,33014.0,"$268,800","$1,892"
7,33015.0,"$271,400","$1,820"
8,33016.0,"$204,900","$1,673"
9,33018.0,"$327,400","$2,172"
10,33027.0,"$317,800","$2,186"
11,33030.0,"$258,600","$1,816"
12,33031.0,"$466,700",---


In [14]:
income_data = income_data[['GEO_ID','S1902_C03_001E']]

income_data

,GEO_ID,S1902_C03_001E
0,id,Estimate!!Mean income (dollars)!!HOUSEHOLD INC...
1,1400000US12086000107,80065
2,1400000US12086000109,51107
3,1400000US12086000113,96580
4,1400000US12086000115,155974
5,1400000US12086000118,292064
6,1400000US12086000119,92746
7,1400000US12086000120,121055
8,1400000US12086000121,145378
9,1400000US12086000122,96652


In [15]:
income_data = income_data.drop(0)

In [16]:
# new data frame with split value columns 
new = income_data["GEO_ID"].str.split("US", n = 1, expand = True) 
  
# making separate first name column from new data frame 
income_data["x"]= new[0] 
  
# making separate last name column from new data frame 
income_data["Tract"]= new[1] 

In [17]:
income_data = income_data[['S1902_C03_001E','Tract']].rename(columns={'S1902_C03_001E':'Avg_income'})
income_data
income_data['Tract']=income_data['Tract'].astype(int)

In [18]:
#income_data = pd.merge(left = income_data,right = zip_vs_tract, left_on='Tract', right_on='tract')
merged_left = pd.merge(left=income_data,right=zip_vs_tract, how='left', left_on='Tract', right_on='tract')

In [19]:
merged_left = merged_left[['Avg_income','zip']]


In [20]:
merged_left

,Avg_income,zip
0,80065,33160.0
1,51107,33160.0
2,51107,33181.0
3,51107,33261.0
4,96580,33163.0
5,96580,33180.0
6,155974,33160.0
7,292064,33160.0
8,92746,33180.0
9,121055,33154.0


In [21]:
merged_left.dropna()
merged_left.Avg_income = pd.to_numeric(merged_left.Avg_income, errors='coerce').fillna(0).astype(int)


In [22]:
merged_left

,Avg_income,zip
0,80065,33160.0
1,51107,33160.0
2,51107,33181.0
3,51107,33261.0
4,96580,33163.0
5,96580,33180.0
6,155974,33160.0
7,292064,33160.0
8,92746,33180.0
9,121055,33154.0


In [23]:
avgincome_perzip = merged_left.groupby(['zip']).mean()


In [24]:
avgincome_perzip

,Avg_income
zip,
33002.0,55473.000000
33010.0,40904.846154
33011.0,32323.000000
33012.0,44724.055556
33013.0,44833.750000
33014.0,61790.416667
33015.0,71307.818182
33016.0,63940.300000
33017.0,42567.000000


In [25]:
#income_homevalue = pd.merge(left=zip_data,right=merged_left, how='inner', left_on='zip_code', right_on='zip')
income_homevalue = pd.merge(left=avgincome_perzip,right=zip_data, how='inner', left_on='zip', right_on='zip_code')

In [26]:
income_homevalue['rule_of_thumb'] = (income_homevalue.Avg_income * 0.3 / 12).round(3)

In [27]:
income_homevalue['rent_value'] = income_homevalue['rent_value'].str.replace('$','').str.replace(',','')
income_homevalue['rent_value'] = pd.to_numeric(income_homevalue.rent_value, errors='coerce').fillna(0).astype(int)

In [28]:
income_homevalue['home_value'] = income_homevalue['home_value'].str.replace('$','').str.replace(',','')
income_homevalue['home_value'] = pd.to_numeric(income_homevalue.home_value, errors='coerce').fillna(0).astype(int)

In [29]:
income_homevalue['disparity'] = income_homevalue.rent_value - income_homevalue.rule_of_thumb

In [30]:
income_homevalue = income_homevalue.sort_values(by='disparity', ascending= False).sort_values(by='Avg_income')

In [31]:
income_homevalue = income_homevalue[income_homevalue.rent_value != 0]

In [37]:
top30_disparity = income_homevalue.head(30)

In [33]:
top30_disparity['zip_code'] = top30_disparity['zip_code'].astype(int)
top30_disparity['Avg_income'] = top30_disparity['Avg_income'].round(3)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
top30_disparity = top30_disparity.reset_index(drop=True)


In [35]:
zip_list = top30_disparity['zip_code'].tolist()

In [36]:
zip_list

[33142,
 33127,
 33147,
 33125,
 33136,
 33012,
 33167,
 33126,
 33150,
 33032,
 33130,
 33162,
 33034,
 33055,
 33056,
 33174,
 33172,
 33033,
 33168,
 33190,
 33169,
 33181,
 33030,
 33014,
 33193,
 33016,
 33161,
 33184,
 33194,
 33035]